In [2]:
import numpy as np
import pandas as pd 

In [3]:
class nn:
    def __init__(self,hidden_layer_size=100,learning_rate=0.01,neurons=20,iterations=60000,epsilon=1e5,activation_function='tan'):
        self.hidden_layer_size=hidden_layer_size
        self.activation_function=activation_function
        self.learning_rate=learning_rate
        self.layer=list()
        self.layer_weights=list()
        self.output_layer=1
        self.iterations=iterations
        self.epsilon=epsilon
        self.neurons=neurons
        
        
    def create_network(self,X):
        #np.random.seed(1) #to have random in between the specific range
        random_weights=2*np.random.random((X.shape[1],self.neurons))-1
        self.layer_weights.append(random_weights)
        for i in range(self.hidden_layer_size-2):
            random_weights=2*np.random.random((self.neurons,self.neurons))-1
            self.layer_weights.append(random_weights)
        random_weights=2*np.random.random((self.neurons,self.output_layer))-1
        self.layer_weights.append(random_weights)
        
        
    def activation(self,x,derivative=False):
        if derivative:
            if self.activation_function == "sigmoid":
                return x * (1 - x)
            if self.activation_function=="tan":
                return 1.0 - np.tanh(x)**2
            if self.activation_function == "ReLU":
                return (x > 0).astype(int)        
        else:
            if self.activation_function == "sigmoid":
                return 1 / (1 + np.exp(-x))
            if self.activation_function=="tan":
                    return np.tanh(x)
            if self.activation_function == "ReLU":
                return x * (x > 0)
            
        
    def fit(self,X,Y):
        end_error=0
        self.create_network(X)
        for _ in range(self.iterations):
            #feed forward throught the network
            self.layer=list()
            self.layer.append(X)
            for i in range(self.hidden_layer_size):
                hidden_layer=self.activation(np.dot(self.layer[i],self.layer_weights[i]))
                self.layer.append(hidden_layer)
            
            error=Y-self.layer[-1]
            end_error=np.mean(np.abs(error))
            if(_%100==1):
                print(str(_)+" Error "+str(end_error))
            for i in range(self.hidden_layer_size,0,-1):
                delta = error*self.activation(self.layer[i],derivative=True)
                error = delta.dot(self.layer_weights[i-1].T)
                self.layer_weights[i-1] += self.layer[i-1].T.dot(delta)
       
        print("End Error"+str(end_error))

    
    def predict(self,X):
        predicted=X
        for i in range(self.hidden_layer_size):
            predicted=self.activation(np.dot(predicted,self.layer_weights[i]))
        return predicted.ravel()
    
    #need to remove
    def sigmoid(self,x,derivative):
            if(derivative==1):
                return 1.0 - np.tanh(x)**2
            return np.tanh(x)
        
    
    def layer3(self,X,Y,X_test):
        y=Y
        phi=X
        rows,col=phi.shape

        num_of_input_neurons=X.shape[1]
        num_of_hidden_layer1_neurons=self.neurons
        num_of_hidden_layer2_neurons=self.neurons
        num_of_output_layer_neurons=1
        
        wt1=2*np.random.random((num_of_input_neurons,num_of_hidden_layer1_neurons))-1
        wt2=2*np.random.random((num_of_hidden_layer1_neurons,num_of_hidden_layer2_neurons))-1
        wt3=2*np.random.random((num_of_hidden_layer2_neurons,num_of_output_layer_neurons))-1
        
        data=phi
        train_y=np.reshape(y,(rows,1))
        train_x= np.array(data)
        
        for j in range(self.iterations):
            layer1_output=self.sigmoid(np.dot(train_x,wt1),0)
            layer2_output=self.sigmoid(np.dot(layer1_output,wt2),0)
            layer3_output=self.sigmoid(np.dot(layer2_output,wt3),0)
            layer3_error=layer3_output-train_y
            layer3_senstivity = layer3_error*self.sigmoid(layer3_output,1)
            layer2_error=layer3_senstivity.dot(wt3.T)
            layer2_senstivity=layer2_error*(self.sigmoid(layer2_output,1))
            layer1_error=layer2_senstivity.dot(wt2.T)
            layer1_senstivity=layer1_error*(self.sigmoid(layer1_output,1))
            wt3=wt3-(self.learning_rate*layer2_output.T.dot(layer3_senstivity))
            wt2=wt2-self.learning_rate*layer1_output.T.dot(layer2_senstivity)
            wt1=wt1-self.learning_rate*(train_x.T).dot(layer1_senstivity)

        layer1= self.sigmoid(np.dot(X_test,wt1),0)
        layer2= self.sigmoid(np.dot(layer1,wt2),0)
        layer3= self.sigmoid(np.dot(layer2,wt3),0)
        layer3=layer3.ravel()
        layer3[layer3 > 0]=1
        layer3[layer3<=0]=0
        return layer3



In [4]:
def normalize(inputData):
    #return (inputData - inputData.mean()) / inputData.std()
    return (inputData - inputData.min()) / (inputData.max() - inputData.min())

#reads the datafiles and returns the training and the testing data
def get_data():
    # get test & test csv files as a DataFrame
    train_df = pd.read_csv("data/train.csv")
    test_df    = pd.read_csv("data/test.csv")

    
    #removing race and native country
    train_df=train_df.drop(['race'],axis=1)
    train_df=train_df.drop(['native-country'],axis=1)
#     train_df=train_df.drop(['occupation'],axis=1)
    train_df=train_df.drop(['relationship'],axis=1)

    test_df=test_df.drop(['race'],axis=1)
    test_df=test_df.drop(['native-country'],axis=1)
#     test_df=test_df.drop(['occupation'],axis=1)
    test_df=test_df.drop(['relationship'],axis=1)

    numericalColumns = ('age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week')
    for i in numericalColumns:
        train_df[i] = normalize(train_df[i])
        test_df[i] = normalize(test_df[i])

    
    #creating dummies of the data
    train_df=pd.get_dummies(train_df)
    test_df=pd.get_dummies(test_df)

    #remove unwanted columns and the columns that are created for ?
    columns_to_remove=set(list(train_df)).symmetric_difference(set(list(test_df)))
    columns_to_remove.remove('salary')
    for col in list(train_df):
        if (col in columns_to_remove) or ("?" in col) :
            print(col)
            train_df=train_df.drop(col,1)
    for col in list(test_df):
        if (col in columns_to_remove) or ("?" in col) :
            test_df=test_df.drop(col,1)
    
    return train_df,test_df


def process_data(percent):
    train_df,test_df=get_data()
    test_ids=test_df['id'].as_matrix()
    train_df=train_df.drop(['id'],1)
    test_df=test_df.drop(['id'],1)
    train_df['const']=1
    test_df['const']=1
    Y=train_df['salary'].as_matrix()
    X=train_df.drop(['salary'], axis=1).as_matrix()
    Y=Y.reshape(len(Y),1)
    end=int(X.shape[0] * percent)
    #training data
    train_X=X[:end,:]
    train_Y=Y[:end,:]
    #data for cross validation
    cross_X=X[end:,:]
    cross_Y=Y[end:,:]
    #testing data
    test_X=test_df.as_matrix()
    
    return train_X,train_Y,cross_X,cross_Y,test_X,test_ids



#writes the predicted values to file 
def write_result(ids,predicted,file_name):
    output=np.column_stack((ids,predicted))
    np.savetxt(file_name,output,delimiter=",",fmt="%d,%d",header="id,salary",comments ='')
    


In [7]:
train_X,train_Y,cross_X,cross_Y,test_X,test_ids= process_data(0.50)
neural_network=nn(hidden_layer_size=3,neurons=30,iterations=5000,learning_rate=0.0001)
neural_network.fit(train_X,train_Y)
predict=neural_network.predict(test_X)
print(predict.shape)
predict[predict>0]=1
predict[predict<=0]=0
print(test_ids.shape,predict.shape)
write_result(test_ids,predict,"hareesh.csv")


workclass_ ?
occupation_ ?
1 Error 0.762906702248
101 Error 0.762906702248
201 Error 1.23709329775
301 Error 0.777686544186
401 Error 0.764035717951
501 Error 0.762906702248
601 Error 0.79636662219
701 Error 1.14369290773
801 Error 0.992917992405
901 Error 1.18628759109
1001 Error 0.770296623217
1101 Error 1.23709329775
1201 Error 0.763830442369
1301 Error 1.23709329775
1401 Error 1.20691778713
1501 Error 1.01324023865
1601 Error 0.988607204029
1701 Error 0.762906702248
1801 Error 0.763009340039
1901 Error 1.23709329775
2001 Error 1.22395566047
2101 Error 0.763419891204
2201 Error 1.23709329775
2301 Error 0.767114851688
2401 Error 1.19511444114
2501 Error 0.767730678436
2601 Error 1.23709329775
2701 Error 0.97505901673
2801 Error 1.18875089808
2901 Error 0.764343631325
3001 Error 0.762906702248
3101 Error 0.762906702248
3201 Error 0.891717130247
3301 Error 1.21656573951
3401 Error 0.767525402853
3501 Error 0.764035717951
3601 Error 0.771630914503
3701 Error 1.22611105409
3801 Error 1.2

In [5]:
t=neural_network.layer3(train_X,train_Y,test_X)
write_result(test_ids,t,"ayush.csv")


In [11]:
predict[predict>0]=1
predict[predict<=0]=0
print(test_ids.shape,predict.shape)
write_result(test_ids,predict,"hareesh.csv")

(6878,) (6878,)


In [ ]:
#playgorund
X= np.array([ [0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1], ])

Y=np.array([ [ 0,1,1,1 ] ]).T

n=nn(hidden_layer_size=1,neurons=20)

predict=n.ayush(X,Y,X)
print(predict)
